<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/no_equal/Continue_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 393 (delta 88), reused 93 (delta 47), pack-reused 243
Receiving objects: 100% (393/393), 6.01 MiB | 1.10 MiB/s, done.
Resolving deltas: 100% (219/219), done.
/content/bachelor_design


In [3]:
!git checkout no_equal
!git pull

Branch 'no_equal' set up to track remote branch 'no_equal' from 'origin'.
Switched to a new branch 'no_equal'
Already up to date.


# Mount GDriver
Perpare Colab environment.

In [4]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


In [5]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none-any.whl size=36139 sha256=7429fdb324616ba537a9dfc7e27b09cea2495a2c934b723a2607b89f58841c30
  Stored in directory: /root/.cache/pip/wheels/1f/59/04/12e95257aebfd27f7edaaf65ab7dd57b5f6cadfb183f1a4ccd
  Created wheel for keras-transformer: filename=keras_transformer-0.37.0-cp36-none-any.whl size=12942 sha256=1386e76f5c9fcbe59edfd177a8dd801069ccefabbd845d6ca5ca99321fe2d921
  Stored in directory: /root/.cache/pip/wheels/2a/f9/31/2a3289e948852ce0dd3fcd94c34bbc7eb9628842cb7110a87b
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=8fdc9562c8f7cf828397f8e9ac38d5d74021bf18dacb38d545dc92ea1f69183b
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15611 sha256=c77295ea831ef2585739f2f5ea12392ba55cfd2f0a7d7e3aca38dd12d94579e4
  Stor

In [6]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-0u1p405k
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-0u1p405k
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=59e0b5038b4d49f98e21724eac3a9e1c673621928611e10e6bc16818db89f3fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-fr87pto5/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")
os.chdir("bachelor_design")

# Load Params

In [0]:
import json
import config

In [0]:
tag_vocab = None
with open(addh + config.TAG_VOCAB_PATH, "r") as fd:
  tag_vocab = json.load(fd)

# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess

In [11]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN,
    tag_vocab=tag_vocab
    )

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.908 seconds.
Prefix dict has been built successfully.
Using TensorFlow backend.


In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [13]:
tag_vocab

{'': 0,
 'B-EI': 2,
 'B-EIF': 8,
 'B-EO': 4,
 'B-EQ': 10,
 'B-ILF': 6,
 'I-EI': 3,
 'I-EIF': 9,
 'I-EO': 5,
 'I-EQ': 11,
 'I-ILF': 7,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [15]:
bert_model, bert_model_config = keras_bert.build_model_from_config(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [18]:
model.load_weights(addh + config.MODEL_PATH)

KeyboardInterrupt: ignored

# Load train record

In [0]:
import json
with open(addh + config.TRAIN_REC_PATH, "r") as fd:
    train_rec = json.load(fd)
last_epoch = 0
if len(train_rec) > 1:
    last_epoch = train_rec[-1]["epoch"]

# Fine-tuning

In [0]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)

In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
batch_size = 16
completed = False
import tensorflow as tf

In [0]:
while not completed:
  try:
    model.fit(
        x=[train_token_id_seqs, train_segment_seqs],
        y=[train_tag_id_seqs], 
        batch_size=batch_size,
        initial_epoch=last_epoch+1,
        epochs=train_rec[0]["train_params"]["epochs"],
        verbose=1,
        validation_data=[[test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]],
        callbacks=[epoch_callback]
    )
    completed = True
  except tf.errors.ResourceExhaustedError as e:
    batch_size = int(batch_size/2)
    print("Batch Size too large, turn to " + str(batch_size))
    if batch_size < 1:
      raise e

In [0]:
model.save(addh + config.MODEL_PATH)